In [1]:
#!/usr/bin/env python
# coding: utf-8

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
5,application_1691454429219_0006,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [1]:
import boto3
import os
import sys
import pytz
import numpy as np
import datetime
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import udf, lpad, translate
from datetime import datetime 
from datetime import timedelta
from datetime import date
from dateutil.relativedelta import relativedelta
from pyspark.sql.types import *
from pyspark.sql.functions import count, avg

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1691628408595_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
agora=datetime.now(pytz.timezone('America/Sao_Paulo'))
dthproc=agora.strftime("%Y%m%d%H%M%S")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
spark = SparkSession.builder.appName('trusted_orders').getOrCreate()
sqlContext=SQLContext(spark.sparkContext)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
_DIA_ = '20230804'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
bucket_refined = 'bkt-datalake-grupo-01-hackathon-podacademy/{dia}/004_refined'.format(dia=_DIA_)
bucket_trusted = 'bkt-datalake-grupo-01-hackathon-podacademy/{dia}/003_trusted'.format(dia=_DIA_)
bucket_raw = 'bkt-datalake-grupo-01-hackathon-podacademy/{dia}/002_raw'.format(dia=_DIA_)
bucket_control = 'bkt-datalake-grupo-01-hackathon-podacademy/{dia}/005_control'.format(dia=_DIA_)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
output_refined = 't_indicadores_variacao_inadimp'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
path_lake = os.path.join('s3://',bucket_trusted,'trusted_bureau_20230805')
bureau_00=sqlContext.read.format("parquet").load(path_lake)
bureau_00.registerTempTable("bureau_00")
bureau_00.cache()
bureau_00.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1716428

In [8]:
bureau_00.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- ref: integer (nullable = true)
 |-- ts_proc: long (nullable = true)
 |-- SK_ID_CURR: integer (nullable = true)
 |-- SK_ID_BUREAU: integer (nullable = true)
 |-- CREDIT_ACTIVE: string (nullable = true)
 |-- CREDIT_CURRENCY: string (nullable = true)
 |-- DAYS_CREDIT: integer (nullable = true)
 |-- CREDIT_DAY_OVERDUE: integer (nullable = true)
 |-- DAYS_CREDIT_ENDDATE: double (nullable = true)
 |-- DAYS_ENDDATE_FACT: double (nullable = true)
 |-- AMT_CREDIT_MAX_OVERDUE: double (nullable = true)
 |-- CNT_CREDIT_PROLONG: integer (nullable = true)
 |-- AMT_CREDIT_SUM: double (nullable = true)
 |-- AMT_CREDIT_SUM_DEBT: double (nullable = true)
 |-- AMT_CREDIT_SUM_LIMIT: double (nullable = true)
 |-- AMT_CREDIT_SUM_OVERDUE: double (nullable = true)
 |-- CREDIT_TYPE: string (nullable = true)
 |-- DAYS_CREDIT_UPDATE: integer (nullable = true)
 |-- AMT_ANNUITY: double (nullable = true)
 |-- ref_partition: integer (nullable = true)
 |-- ts_proc_partition: long (nullable = true)

In [9]:
path_lake = os.path.join('s3://',bucket_trusted,'trusted_bureau_balance_20230805/')
bureau_balance_00=sqlContext.read.format("parquet").load(path_lake)
bureau_balance_00.registerTempTable("bureau_balance_00")
bureau_balance_00.cache()
bureau_balance_00.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

27299925

In [10]:
bureau_balance_00.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- ref: integer (nullable = true)
 |-- ts_proc: long (nullable = true)
 |-- SK_ID_BUREAU: integer (nullable = true)
 |-- MONTHS_BALANCE: integer (nullable = true)
 |-- STATUS: string (nullable = true)
 |-- ref_partition: integer (nullable = true)
 |-- ts_proc_partition: long (nullable = true)

In [18]:
indicadores_00 = spark.sql(     
    """
        SELECT
            SK_ID_BUREAU,
            MONTHS_BALANCE*(-1) AS MONTHS_BALANCE,
            CASE
                WHEN MONTHS_BALANCE BETWEEN -3 AND 0 THEN '0-3'
                WHEN MONTHS_BALANCE BETWEEN -6 AND -4 THEN '4-6'
                WHEN MONTHS_BALANCE BETWEEN -9 AND -7 THEN '7-9'
                WHEN MONTHS_BALANCE BETWEEN -12 AND -10 THEN '10-12'
                WHEN MONTHS_BALANCE BETWEEN -13 AND -24 THEN '13-24'
                ELSE '24+'
                END AS BUCKET_MONTHS                
        FROM
            bureau_balance_00
        WHERE 
            MONTHS_BALANCE < 0
    """)
indicadores_00.registerTempTable("indicadores_00")
indicadores_00.cache()
indicadores_00.count() 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

26688960

In [19]:
indicadores_00.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+--------------+-------------+
|SK_ID_BUREAU|MONTHS_BALANCE|BUCKET_MONTHS|
+------------+--------------+-------------+
|     6608633|            62|          24+|
|     5715448|             1|          0-3|
|     6608633|            63|          24+|
|     5715448|             2|          0-3|
|     6608633|            64|          24+|
|     5715448|             3|          0-3|
|     6608633|            65|          24+|
|     5715448|             4|          4-6|
|     6608633|            66|          24+|
|     5715448|             5|          4-6|
|     6608633|            67|          24+|
|     5715448|             6|          4-6|
|     6608633|            68|          24+|
|     5715448|             7|          7-9|
|     6608633|            69|          24+|
|     5715448|             8|          7-9|
|     6608633|            70|          24+|
|     5715448|             9|          7-9|
|     6608633|            71|          24+|
|     5715448|            10|   

In [20]:
indicadores_01 = spark.sql(     
    """
        SELECT 
            bureau_00.SK_ID_BUREAU,
            bureau_00.CREDIT_ACTIVE,
            bureau_00.AMT_CREDIT_SUM,
            bureau_00.AMT_CREDIT_SUM_OVERDUE,            
            indicadores_00.MONTHS_BALANCE,
            indicadores_00.BUCKET_MONTHS
        FROM 
            indicadores_00
        LEFT JOIN 
            bureau_00 
        ON indicadores_00.SK_ID_BUREAU = bureau_00.SK_ID_BUREAU 
        WHERE
            bureau_00.CREDIT_ACTIVE = 'Active'

    """)
indicadores_01.registerTempTable("indicadores_01")
indicadores_01.cache()
indicadores_01.count() 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

5305733

In [21]:
indicadores_01.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+-------------+--------------+----------------------+--------------+-------------+
|SK_ID_BUREAU|CREDIT_ACTIVE|AMT_CREDIT_SUM|AMT_CREDIT_SUM_OVERDUE|MONTHS_BALANCE|BUCKET_MONTHS|
+------------+-------------+--------------+----------------------+--------------+-------------+
|     5001812|       Active|      135000.0|                   0.0|             1|          0-3|
|     5001812|       Active|      135000.0|                   0.0|             2|          0-3|
|     5001812|       Active|      135000.0|                   0.0|             3|          0-3|
|     5001812|       Active|      135000.0|                   0.0|             4|          4-6|
|     5001812|       Active|      135000.0|                   0.0|             5|          4-6|
|     5001812|       Active|      135000.0|                   0.0|             6|          4-6|
|     5001812|       Active|      135000.0|                   0.0|             7|          7-9|
|     5001812|       Active|      135000

In [22]:
indicadores_02 = spark.sql(     
    """
        SELECT 
            SK_ID_BUREAU,
            MAX(MONTHS_BALANCE) AS ULTIMO_MONTH_BALANCE
        FROM 
            indicadores_01
        GROUP BY
            SK_ID_BUREAU
    """)
indicadores_02.registerTempTable("indicadores_02")
indicadores_02.cache()
indicadores_02.count() 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

277388

In [23]:
indicadores_02.show(100)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+--------------------+
|SK_ID_BUREAU|ULTIMO_MONTH_BALANCE|
+------------+--------------------+
|     5001812|                  22|
|     5003338|                  33|
|     5003386|                   7|
|     5003485|                  16|
|     5004511|                  49|
|     5004620|                  16|
|     5005000|                   6|
|     5009304|                  34|
|     5009355|                  25|
|     5010070|                  10|
|     5010236|                  14|
|     5010338|                   2|
|     5010511|                  11|
|     5011188|                  84|
|     5011347|                   9|
|     5012597|                   7|
|     5021944|                  16|
|     5021994|                   4|
|     5022024|                  22|
|     5023675|                   3|
|     5024412|                  11|
|     5024771|                  15|
|     5025981|                  19|
|     5028624|                   7|
|     5029350|              

In [24]:
indicadores_03 = spark.sql(     
    """
        SELECT 
            indicadores_02.SK_ID_BUREAU,
            indicadores_01.AMT_CREDIT_SUM,
            indicadores_01.AMT_CREDIT_SUM_OVERDUE,
            indicadores_02.ULTIMO_MONTH_BALANCE,
            indicadores_01.BUCKET_MONTHS
        FROM 
            indicadores_02
        LEFT JOIN 
            indicadores_01
        ON 
            indicadores_02.SK_ID_BUREAU = indicadores_01.SK_ID_BUREAU
        AND 
            indicadores_02.ULTIMO_MONTH_BALANCE = indicadores_01.MONTHS_BALANCE
    """)
indicadores_03.registerTempTable("indicadores_03")
indicadores_03.cache()
indicadores_03.count() 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

277388

In [25]:
indicadores_03.show(50)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+--------------+----------------------+--------------------+-------------+
|SK_ID_BUREAU|AMT_CREDIT_SUM|AMT_CREDIT_SUM_OVERDUE|ULTIMO_MONTH_BALANCE|BUCKET_MONTHS|
+------------+--------------+----------------------+--------------------+-------------+
|     5004005|      765000.0|                   0.0|                  19|          24+|
|     5004464|       56250.0|                   0.0|                  27|          24+|
|     5009036|       56866.5|                   0.0|                  18|          24+|
|     5009521|           0.0|                   0.0|                  32|          24+|
|     5009639|      684000.0|                   0.0|                  39|          24+|
|     5009826|     441783.36|                   0.0|                  25|          24+|
|     5010499|     2601000.0|                   0.0|                  56|          24+|
|     5011515|      153000.0|                   0.0|                  11|        10-12|
|     5012706|     2380500.0|   

In [26]:
indicadores_04 = spark.sql(     
    """
        SELECT 
            SUM(AMT_CREDIT_SUM) AS SUM_AMT_CREDIT_SUM,
            SUM(AMT_CREDIT_SUM_OVERDUE) AS SUM_AMT_CREDIT_SUM_OVERDUE,
            BUCKET_MONTHS
        FROM 
            indicadores_03
        GROUP BY
            BUCKET_MONTHS
        ORDER BY
            BUCKET_MONTHS
    """)
indicadores_04.registerTempTable("indicadores_04")
indicadores_04.cache()
indicadores_04.count() 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

5

In [27]:
indicadores_04.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------------+-------------+
|  SUM_AMT_CREDIT_SUM|SUM_AMT_CREDIT_SUM_OVERDUE|BUCKET_MONTHS|
+--------------------+--------------------------+-------------+
| 8.010311455530001E9|        461633.98500000004|          0-3|
|  1.7286224921055E10|               1839590.865|        10-12|
|  9.7437423435822E10|         9866368.260000007|          24+|
|1.625650653091499...|                1420445.61|          4-6|
|1.849223973379499...|        1260122.8050000002|          7-9|
+--------------------+--------------------------+-------------+

In [28]:
indicadores_05 = spark.sql(     
    """
        SELECT 
            SUM_AMT_CREDIT_SUM_OVERDUE / SUM_AMT_CREDIT_SUM AS TAXA_INADIMPLENCIA,
            BUCKET_MONTHS
        FROM 
            indicadores_04
        ORDER BY
            BUCKET_MONTHS
    """)
indicadores_05.registerTempTable("indicadores_05")
indicadores_05.cache()
indicadores_05.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

5

In [29]:
indicadores_05.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------+
|  TAXA_INADIMPLENCIA|BUCKET_MONTHS|
+--------------------+-------------+
|5.762996701973508E-5|          0-3|
|1.064194682992547...|        10-12|
|1.012585094319389E-4|          24+|
|8.737705160076914E-5|          4-6|
|6.814333056136497E-5|          7-9|
+--------------------+-------------+

In [30]:
tabela = output_refined

path_athena = os.path.join('s3://',bucket_refined,tabela)
print('path_tabela:', path_athena)
indicadores_05.coalesce(1).write.\
format("parquet").mode("overwrite").option("compression", "gzip").option("overwriteSchema", "true").\
option("path", path_athena).saveAsTable(tabela)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

path_tabela: s3://bkt-datalake-grupo-01-hackathon-podacademy/20230804/004_refined/t_indicadores_variacao_inadimp